# Query the data

## Get keys and urls

In [26]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
JINAAI_API_KEY = os.getenv("JINAAI_API_KEY")

print(WEAVIATE_URL)
print(WEAVIATE_KEY)
print(JINAAI_API_KEY)

if(WEAVIATE_URL == "UPDATE_ME_WEAVIATE_URL"):
    raise Exception("Please update .env and Restart the notebook (see Restart button, next to Run All)")

https://vm7vt6fttjqqoh2riw9mvg.c0.europe-west3.gcp.weaviate.cloud
6VxSJTcUZ7GY0l986XYFTt3mkEMeIm7F0pNc
jina_0dbcfcbf17fe431aa4e4ebb2d7e31a6caI4V-9CeQnNfQcpnqys_hQzR4f3y


## Connect to Weaviate

In [31]:
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    headers = {
        "X-JinaAI-Api-Key": JINAAI_API_KEY,
    },

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=30, query=120, insert=120)  # Values in seconds
    #)
    additional_config=AdditionalConfig(
         timeout=Timeout(init=2, query=120, insert=120),  # Values in seconds
    )
)

client.is_ready()

True

### Helper function

In [32]:
import json
def print_properties(item):
    print(
        json.dumps(
            item.properties,
            indent=2, sort_keys=True, default=str
        )
    )

## Vector search
[Docs - near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

In [34]:
# TODO: get the Wiki collection
# wiki = 
wiki = client.collections.get("Wiki")

# TODO: run a near text query, search for musical instruments, with limit 5
# response = wiki.query.
response = wiki.query.near_text("musikinstrumente", limit=5 )
# response = wiki.query.near_text("music instruments", limit=5 )
for item in response.objects:
    print_properties(item)

{
  "text": "Bagpipes have always been folk instruments, but after the 15th century some were used for court music, and others have survived as military instruments. For the chanter, two single-reed cane pipes are placed parallel, one pipe often sounding a drone or other accompaniment to the other pipe.Oct 6, 2023.",
  "title": "Bagpipes",
  "url": "https://simple.wikipedia.org/wiki/Bagpipes",
  "wiki_id": "20231101.simple_112374_3"
}
{
  "text": "A shepherd had a lot of time to sit and think or amuse himself.  He often made simple instruments out of reeds that grew where he sat.  These instruments were often simple pan flutes or just pipes which were similar to bamboo pipes or a modern recorder.  Some shepherds learned to carve instruments out of wood.  An example is the fujara from Slovakia and southern Poland.",
  "title": "Shepherd",
  "url": "https://simple.wikipedia.org/wiki/Shepherd",
  "wiki_id": "20231101.simple_70678_5"
}
{
  "text": "An Aeolian harp is just a box with tuned 

In [36]:
from weaviate.classes.query import MetadataQuery

## wiki = client.collections.get(WIKI_NAME)

response = wiki.query.near_text(
    query="musical instruments",
    # limit=5,
    auto_limit=2,
    distance=0.361,
    # TODO: add MetadataQuery - request distance
    # return_metadata=
    return_metadata=MetadataQuery(distance=True)
)


for item in response.objects:
    print_properties(item)
    print(item.metadata.distance)

{
  "text": "Bagpipes have always been folk instruments, but after the 15th century some were used for court music, and others have survived as military instruments. For the chanter, two single-reed cane pipes are placed parallel, one pipe often sounding a drone or other accompaniment to the other pipe.Oct 6, 2023.",
  "title": "Bagpipes",
  "url": "https://simple.wikipedia.org/wiki/Bagpipes",
  "wiki_id": "20231101.simple_112374_3"
}
0.3550281524658203


### Autocut

Return groups of results based on the quality/distance jumps

In [37]:
from weaviate.classes.query import MetadataQuery

#wiki = client.collections.get(WIKI_NAME)

response = wiki.query.near_text(
    query="musical instruments",
    # query="techno",
    auto_limit=1,
    # TODO: use auto_limit instead if limit, set it to 1

    return_metadata=MetadataQuery(distance=True)
)

print(f"Returned object count: {len(response.objects)}")

for item in response.objects:
    print_properties(item)
    print(item.metadata.distance)

Returned object count: 2
{
  "text": "Bagpipes have always been folk instruments, but after the 15th century some were used for court music, and others have survived as military instruments. For the chanter, two single-reed cane pipes are placed parallel, one pipe often sounding a drone or other accompaniment to the other pipe.Oct 6, 2023.",
  "title": "Bagpipes",
  "url": "https://simple.wikipedia.org/wiki/Bagpipes",
  "wiki_id": "20231101.simple_112374_3"
}
0.3550281524658203
{
  "text": "The simplest type of mechanical aid is the use of wooden \"fingers\" that can be closed to press the string down and fret the C#, D, Eb, or E notes. This system is particularly useful for basslines that have a repeating pedal point such as a low D, because once the note is locked in place with the mechanical \"finger\", the lowest string then sounds a different note when it is played \"open\" (e.g., a low D).",
  "title": "Double bass",
  "url": "https://simple.wikipedia.org/wiki/Double%20bass",
  "

## Filters

### Fetch with filters

In [41]:
from weaviate.classes.query import Filter

# wiki = client.collections.get(WIKI_NAME)

response = wiki.query.fetch_objects(
    limit=5,
    # TODO: filter by property title, search for something like m_sic without music
    # filters=
    filters=Filter.by_property("title").like("m*ic") 
        & Filter.by_property("title").not_equal("music")
    , return_metadata=MetadataQuery(distance=True)
)

for item in response.objects:
    print_properties(item)

{
  "text": "In 1886, Adolf Mayer first described the tobacco mosaic disease that could be transferred between plants, similar to bacterial infections.",
  "title": "Tobacco mosaic virus",
  "url": "https://simple.wikipedia.org/wiki/Tobacco%20mosaic%20virus",
  "wiki_id": "20231101.simple_528724_2"
}
{
  "text": "By its very nature, the evidence for this idea comes mainly from palaeontology. It is not claimed that this pattern is universal, but it is common. There are now a wide range of examples from many different taxa.",
  "title": "Mosaic evolution",
  "url": "https://simple.wikipedia.org/wiki/Mosaic%20evolution",
  "wiki_id": "20231101.simple_290228_4"
}
{
  "text": "\"Magic\" by Pilot is used, its 3rd use in a movie soundtrack after Disney's Herbie Fully Loaded and Happy Gilmore",
  "title": "The Magic Roundabout (movie)",
  "url": "https://simple.wikipedia.org/wiki/The%20Magic%20Roundabout%20%28movie%29",
  "wiki_id": "20231101.simple_117801_2"
}
{
  "text": "What enables the Kr

In [17]:
from weaviate.classes.query import Filter

response = wiki.query.fetch_objects(
    limit=5,
    # NOTE: you can use & as AND operator and | as OR operator
    filters=Filter.by_property("title").like("m*ic") & Filter.by_property("title").not_equal("music")
)

for item in response.objects:
    print_properties(item)

{
  "text": "In 1886, Adolf Mayer first described the tobacco mosaic disease that could be transferred between plants, similar to bacterial infections.",
  "title": "Tobacco mosaic virus",
  "url": "https://simple.wikipedia.org/wiki/Tobacco%20mosaic%20virus",
  "wiki_id": "20231101.simple_528724_2"
}
{
  "text": "By its very nature, the evidence for this idea comes mainly from palaeontology. It is not claimed that this pattern is universal, but it is common. There are now a wide range of examples from many different taxa.",
  "title": "Mosaic evolution",
  "url": "https://simple.wikipedia.org/wiki/Mosaic%20evolution",
  "wiki_id": "20231101.simple_290228_4"
}
{
  "text": "\"Magic\" by Pilot is used, its 3rd use in a movie soundtrack after Disney's Herbie Fully Loaded and Happy Gilmore",
  "title": "The Magic Roundabout (movie)",
  "url": "https://simple.wikipedia.org/wiki/The%20Magic%20Roundabout%20%28movie%29",
  "wiki_id": "20231101.simple_117801_2"
}
{
  "text": "What enables the Kr

## Search with filters
[Docs - Filters](https://weaviate.io/developers/weaviate/search/filters)

In [42]:
from weaviate.classes.query import Filter

response = wiki.query.near_text(
    query="musical instruments",
    limit=4,
    filters=Filter.by_property("title").not_equal("music")
    , return_metadata=MetadataQuery(distance=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.distance)

{
  "text": "Bagpipes have always been folk instruments, but after the 15th century some were used for court music, and others have survived as military instruments. For the chanter, two single-reed cane pipes are placed parallel, one pipe often sounding a drone or other accompaniment to the other pipe.Oct 6, 2023.",
  "title": "Bagpipes",
  "url": "https://simple.wikipedia.org/wiki/Bagpipes",
  "wiki_id": "20231101.simple_112374_3"
}
0.3550281524658203
{
  "text": "The simplest type of mechanical aid is the use of wooden \"fingers\" that can be closed to press the string down and fret the C#, D, Eb, or E notes. This system is particularly useful for basslines that have a repeating pedal point such as a low D, because once the note is locked in place with the mechanical \"finger\", the lowest string then sounds a different note when it is played \"open\" (e.g., a low D).",
  "title": "Double bass",
  "url": "https://simple.wikipedia.org/wiki/Double%20bass",
  "wiki_id": "20231101.simpl

## Keyword Search

[Docs - keyword/bm25](https://weaviate.io/developers/weaviate/search/bm25)

In [50]:
# TODO: use bm25 query, search for musical instruments, set limit to 5

# response = wiki.

response = wiki.query.bm25(
    query="musical instruments",
    limit=5,
    filters=Filter.by_property("title").not_equal("music"),
    # return_metadata=MetadataQuery(distance=True)
    # return_metadata=MetadataQuery(score=True)
    return_metadata=MetadataQuery(distance=True, score=True)
 
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.distance)
    print(item.metadata.score)

{
  "text": "Thousands of years ago people used the horns of dead animals for musical instruments (see horn (postal horn)). In later times, they were used for carrying gunpowder.",
  "title": "Horn (anatomy)",
  "url": "https://simple.wikipedia.org/wiki/Horn%20%28anatomy%29",
  "wiki_id": "20231101.simple_43954_2"
}
None
3.833294153213501
{
  "text": "Cats premiered at the New London Theatre in the West End on 11 May 1981. It closed on 11 May 2002 after 8,949 performances. It opened on Broadway on 7 October 1982 at the Winter Garden Theatre. It closed on 10 September 2000, after 7,485 performances.",
  "title": "Cats (musical)",
  "url": "https://simple.wikipedia.org/wiki/Cats%20%28musical%29",
  "wiki_id": "20231101.simple_70788_1"
}
None
3.671945333480835
{
  "text": "Equal temperament (a special case of well-temperament), in which notes of the scale which are next to one another are all separated by logarithmically equal distances, which are integer powers of 21/12.",
  "title": "Mu

In [51]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.bm25(
    query="musical instruments",
    # TODO: add query properties for "text" and "title"
    # query_properties=[],
    limit=5,
    return_metadata=MetadataQuery(score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)

{
  "text": "Thousands of years ago people used the horns of dead animals for musical instruments (see horn (postal horn)). In later times, they were used for carrying gunpowder.",
  "title": "Horn (anatomy)",
  "url": "https://simple.wikipedia.org/wiki/Horn%20%28anatomy%29",
  "wiki_id": "20231101.simple_43954_2"
}
3.833294153213501
{
  "text": "Cats premiered at the New London Theatre in the West End on 11 May 1981. It closed on 11 May 2002 after 8,949 performances. It opened on Broadway on 7 October 1982 at the Winter Garden Theatre. It closed on 10 September 2000, after 7,485 performances.",
  "title": "Cats (musical)",
  "url": "https://simple.wikipedia.org/wiki/Cats%20%28musical%29",
  "wiki_id": "20231101.simple_70788_1"
}
3.671945333480835
{
  "text": "Equal temperament (a special case of well-temperament), in which notes of the scale which are next to one another are all separated by logarithmically equal distances, which are integer powers of 21/12.",
  "title": "Musical tuni

In [58]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.bm25(
    query="musical instruments",
    query_properties=["text", "title^5"],
    limit=5,
    return_metadata=MetadataQuery(score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)

WeaviateQueryError: Query call with protocol GRPC search failed with message <AioRpcError of RPC that terminated with:
	status = StatusCode.DEADLINE_EXCEEDED
	details = "Deadline Exceeded"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"Deadline Exceeded", grpc_status:4, created_time:"2024-09-28T15:46:37.698836711+00:00"}"
>.

## Hybrid search
[Docs - hybrid](https://weaviate.io/developers/weaviate/search/hybrid)

In [57]:
# TODO: use hybrid query, search for musical instruments, set alpha to 0.7, and limit to 5
# response = wiki.


response = wiki.query.hybrid(
    query="musical notes",
    alpha=0.7,
    limit=5,
    query_properties=["text", "title"],
    # TODO: add query properties for "title"
)
for item in response.objects:
    print_properties(item)

WeaviateQueryError: Query call with protocol GRPC search failed with message <AioRpcError of RPC that terminated with:
	status = StatusCode.DEADLINE_EXCEEDED
	details = "Deadline Exceeded"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2024-09-28T15:39:54.958668032+00:00", grpc_status:4, grpc_message:"Deadline Exceeded"}"
>.

### Hybrid - select properties

In [ ]:
response = wiki.query.hybrid(
    query="musical instruments",
    alpha=0.7,
    limit=5,
    # TODO: add query properties for "title"
)

for item in response.objects:
    print_properties(item)

### Hybrid - Explain score

In [ ]:
from weaviate.classes.query import MetadataQuery

response = wiki.query.hybrid(
    query="musical instruments",
    alpha=0.7,
    limit=5,
    query_properties=["title"],
    return_metadata=MetadataQuery(score=True, explain_score=True)
)

for item in response.objects:
    print_properties(item)
    print(item.metadata.score)
    print(item.metadata.explain_score)

## Close the client

In [24]:
client.close()